# Tracing Custom Dummy App with OTEL Spans using _TruLens_

This notebook demonstrates the "otel-tracing" experimental feature in _TruLens_.
This enables the collection of _OpenTelemetry_ spans during app evaluation. Data
that is collected by _TruLens_ is recorded as spans.

- OTEL exporters demonstrated in this notebook are:

  - Console exporter (prints exported spans in the console or stream). We
    configure this exporter to write spans to a variable `stream`.

  - _Zipkin_ exporter. Setup below includes `docker` commands to download and
    start a _Zipkin_ collector for demonstration purposes. To open the UI for
    this exporter, open _Docker Desktop_, click on the triple dots under
    "Actions" for the zipkin container and select "Open with browser".

In [ ]:
# Install OTEL deps:
# ! pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp

# Install zipkin python package:
# ! pip install opentelemetry-exporter-zipkin-proto-http

# Start the zipkin docker container:
# ! docker run --rm -d -p 9411:9411 --name zipkin openzipkin/zipkin

# Stop the zipkin docker container:
# ! docker stop $(docker ps -a -q --filter ancestor=openzipkin/zipkin)

In [ ]:
# ruff: noqa: F401

from io import StringIO
import json
from pathlib import Path
import re
import sys

import dotenv
from opentelemetry import trace

# zipkip exporter
from opentelemetry.exporter.zipkin.json import ZipkinExporter

# console exporter
from opentelemetry.sdk.trace.export import ConsoleSpanExporter
from trulens.apps.custom import TruCustomApp
from trulens.core import Feedback
from trulens.core import Select
from trulens.core.session import TruSession
from trulens.experimental.otel_tracing.core.trace import TracerProvider
from trulens.feedback.dummy.provider import DummyProvider

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))


dotenv.load_dotenv()

In [ ]:
# Sets the global default tracer provider to be the trulens one.
trace.set_tracer_provider(TracerProvider())

# Creates a tracer for custom spans below.
tracer = trace.get_tracer(__name__)

In [ ]:
# Setup console/file/string exporter
# stream = StringIO()
# Will print A LOT to stdout unless we set a different stream.
# exporter = ConsoleSpanExporter(out=stream)

# Setup zipkin exporter
exporter = ZipkinExporter(endpoint="http://localhost:9411/api/v2/spans")

# Setup session with exporter.
session = TruSession(_experimental_otel_exporter=exporter)

# If not using the exporter, manually enable the otel experimental feature:
# session = TruSession()
# session.experimental_enable_feature("otel_tracing")

session.reset_database()
session.start_dashboard()

In [ ]:
from examples.dev.dummy_app.app import DummyApp

# Create dummy endpoint for a dummy feedback function:
dummy_provider = DummyProvider()
dummy_feedback = Feedback(dummy_provider.sentiment).on(
    text=Select.RecordSpans.trulens.call.generate.attributes[
        "trulens.bindings"
    ].prompt
)
# Parts of the selector are:
#
# - Select.RecordSpans - Select spans dictionary, organized by span name.
#
# - trulens.call.generate - Span name. TruLens call spans are of the form
#   "trulens.call.<method_name>".
#
# - attributes - Span attributes
#
# - ["trulens.bindings"] - Attributes specific to TruLens spans. Call spans include
#   method call arguments in "trulens.bindings".
#
# - prompt - The prompt argument to the method call named.

# Create custom app:
ca = DummyApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
    feedbacks=[dummy_feedback],
)

ca.respond_to_query("hello")

In [ ]:
# Normal trulens recording context manager:
with ta as recorder:
    # Another custom span.
    with tracer.start_as_current_span("custom inner span") as inner_span:
        inner_span.set_attribute("custom", "value")

        # Normal instrumented call:
        print(ca.respond_to_query("hello"))

record = recorder.get()

In [ ]:
# Check the feedback results. Note that this feedback function is from a dummy
# provider which does not have true sentiment analysis.

record.feedback_results[0].result()

In [ ]:
# Check trulens instrumented calls as spans:

record.get(Select.RecordSpans)

In [ ]:
# All of the spans listed above should be visible in the chosen exporter.
#
# The ConsoleSpanExporter writes json dumps of each span. Lets read those back
# here to inspect them:

match_root_json = re.compile(r"(?:(^|\n))\{.+?\n\}", re.DOTALL)

if "stream" in locals():
    dumps = match_root_json.finditer(stream.getvalue())  # noqa: F821

    for dump in dumps:
        span = json.loads(dump.group())

        print(span["name"])